In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob

import numpy as np
import pandas as pd
import os

In [2]:
prefix = '/data/kylevigil/'

# Data Cleaning

In this notebook I will clean up the data collected by the MovieLens group in order to be used by my project.

Here are the tables available in the dataset:

In [3]:
!ls /data/kylevigil

colors.pkl	     movie_locations.dat  user_ratedmovies.dat
genres.pkl	     movies.dat		  user_ratedmovies-timestamps.dat
movie_actors.dat     moviesData.pkl	  user_taggedmovies.dat
movie_countries.dat  movie_tags.dat	  user_taggedmovies-timestamps.dat
movie_directors.dat  readme.txt
movie_genres.dat     tags.dat


## Movies

I will start by importing the table of movies. I will only use the title, url of the movie poster, the number of ratings, and the overall rating of the movie

In [4]:
movies = pd.read_csv(prefix + 'movies.dat', delimiter = '\t', encoding = 'ISO-8859-1', index_col=0)
movies = movies[['title','imdbPictureURL','year','rtAudienceNumRatings','rtAudienceScore']]

In [5]:
movies.head()

,title,imdbPictureURL,year,rtAudienceNumRatings,rtAudienceScore
id,,,,,
1,Toy story,http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,102338,81
2,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,44587,61
3,Grumpy Old Men,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993,10489,66
4,Waiting to Exhale,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995,5666,79
5,Father of the Bride Part II,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995,13761,64


I can extract some data from both the title and the movie poster URL. With the title, I will find the sentiment with the theory that happier movies like childrens movies and romantic movies will have a more positive sentiment than movies like horror movies.

In [6]:
movies['titleSentiment'] = [TextBlob(i).sentiment.polarity for i in movies.title]

From each movie poster I extracted the rgb value of the most popular color. For each rgb color I found the least squared distance to a dictionary of colors that I assembled so I could categorize each color. I then saved these colors to a pickle file called colors.pkl. I go into this in more detail in the notebook: [Color Clusters](02a-ColorClusters.ipynb).

In [7]:
colors = pd.read_pickle(prefix + 'colors.pkl')
colors['colorNum'] = pd.Categorical(colors.posterColor).codes
colors.head()

,posterColor,color,colorNum
id,,,
1,dark red,"(104.392166836, 54.7272562127, 43.522889115)",5
2,tan,"(139.494224539, 107.210170447, 69.5860332441)",10
3,white,"(240.257754103, 245.865108686, 244.831228342)",11
4,white,"(245.888900917, 241.470051238, 222.634877679)",11
5,dark red,"(66.3822844645, 58.6215606039, 44.2912374771)",5


In [8]:
movies.head()

,title,imdbPictureURL,year,rtAudienceNumRatings,rtAudienceScore,titleSentiment
id,,,,,,
1,Toy story,http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,102338,81,0.0
2,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,44587,61,0.0
3,Grumpy Old Men,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993,10489,66,0.1
4,Waiting to Exhale,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995,5666,79,0.0
5,Father of the Bride Part II,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995,13761,64,0.0


In [9]:
assert pd.notnull(movies.title.any())
assert pd.notnull(movies.year.any())
assert len(movies[(movies.titleSentiment > 1) | (movies.titleSentiment < -1)]) == 0

## Directors

Now that the movies table is finished, I will move on to the directors

The only columns that I care about in this table is the directors name along with the movie id.

In [10]:
directors = pd.read_csv(prefix + 'movie_directors.dat', delimiter = '\t', encoding = 'ISO-8859-1', index_col=0)
directors = directors[['directorName']]

I found that the only way I can really use this data is if I make this a categorical set of data. Because of that, I assign a number to each director.

In [11]:
directors['directorNum'] = pd.Categorical(directors.directorName).codes
directors.head()

,directorName,directorNum
movieID,,
1,John Lasseter,2027
2,Joe Johnston,1938
3,Donald Petrie,1038
4,Forest Whitaker,1237
5,Charles Shyer,609


In [12]:
assert pd.notnull(directors.directorName.any())
assert len(directors) > directors.directorNum.max()

## Actors

The next table I moved to was the actors. This along with the directors will hopefully group the movies because actors and directors tend to work in the same genre most of their career.

This table has a list of many actors for each movie ranked by role importance so for simplicity sake I took only the lead actor for each movie.

In [13]:
actors = pd.read_csv(prefix + 'movie_actors.dat', delimiter = '\t', encoding = 'ISO-8859-1', index_col=0)
actors = actors[actors.ranking == 1]
actors = actors[['actorName']]

I also made these into categories by assigning each a number.

In [14]:
actors['actorNum'] = pd.Categorical(actors.actorName.astype('category')).codes
actors.head()

,actorName,actorNum
movieID,,
1,Tom Hanks,7241
2,Robin Williams,6293
3,Jack Lemmon,3122
4,Whitney Houston,7557
5,Steve Martin,6881


In [15]:
assert pd.notnull(actors.actorName.any())
assert len(actors) > actors.actorNum.max()

## Tags

Each movie has user submitted tags that I figured I could take advantage of with the same strategy of the movie title using sentiment analysis.

the tags were listed with one per row as well as being separated between two tables so I needed to gather all tags per movie using groupby operations.

In [16]:
tagNums = pd.read_csv(prefix + 'movie_tags.dat', delimiter = '\t', encoding = 'ISO-8859-1', index_col=0)
tagNums.head()

,tagID,tagWeight
movieID,,
1,7,1
1,13,3
1,25,3
1,55,3
1,60,1


In [17]:
tagText = pd.read_csv(prefix + 'tags.dat', delimiter = '\t', encoding = 'ISO-8859-1', index_col=0)
tagText.head()

,value
id,
1,earth
2,police
3,boxing
4,painter
5,whale


In [18]:
tags = pd.merge(tagNums, tagText, left_on = 'tagID', right_index = True)
tags.head()

,tagID,tagWeight,value
movieID,,,
1,7,1,funny
50,7,1,funny
186,7,1,funny
256,7,1,funny
272,7,1,funny


The only column I needed of these is the value column with the actual text of the tag

In [19]:
tags = tags[['value']]

I then group all of the tags together, space separated.

In [20]:
tags['allTags'] = tags.groupby(tags.index, squeeze = True).agg(lambda x: ' '.join(x.tolist()))
tags = tags.groupby(tags.index).first()
tags = tags[['allTags']]
tags.head()

,allTags
movieID,
1,funny time travel tim allen comedy fun fantasy...
2,time travel fantasy animals children game robi...
3,funniest movies sequel fever jack lemmon walte...
5,family sequel fever sequel remake wedding preg...
6,imdb top 250 witty want bibliothek robert de n...


Now I add the actual data of the sentiment of the tags

In [21]:
tags['tagSentiment'] = [TextBlob(i).sentiment.polarity for i in tags.allTags]
tags.head()

,allTags,tagSentiment
movieID,,
1,funny time travel tim allen comedy fun fantasy...,0.356944
2,time travel fantasy animals children game robi...,-0.337500
3,funniest movies sequel fever jack lemmon walte...,0.100000
5,family sequel fever sequel remake wedding preg...,0.000000
6,imdb top 250 witty want bibliothek robert de n...,0.060606


## Countries

There is also a table with the countries that the movie is released in. I am skeptical of the importance of this data but I might as well include it.

In [22]:
countries = pd.read_csv(prefix + 'movie_countries.dat', delimiter = '\t', encoding = 'ISO-8859-1', index_col=0)

In [23]:
countries['countryNum'] = pd.Categorical(countries.country.astype('category')).codes
countries.head()

,country,countryNum
movieID,,
1,USA,66
2,USA,66
3,USA,66
4,USA,66
5,USA,66


In [24]:
assert pd.notnull(countries.country.any())
assert len(countries) > countries.countryNum.max()

## Genre

Now for the data that I am trying to predict. This was tricky because each movie had been given a list of genres that had up to seven values in it. I reduced these down to one overarching genre in the notebook: [Genre](02b-Genre.ipynb) and save them to a pickle called genres.pkl.

I read in the file and assign each genre a number to categorize them.

In [25]:
genres = pd.read_pickle(prefix + 'genres.pkl')
genres['genreNum'] = pd.Categorical(genres['realGenre']).codes
genres.head()

,realGenre,genreNum
movieID,,
1,Comedy,1
2,None,4
3,Romance,5
4,Romance,5
5,Comedy,1


In [26]:
assert genres.genreNum.max() == 5

## Merging Data

I now need to merge all of the usable data into a single table called moviesData

In [27]:
moviesData = pd.merge(movies[['title','year','rtAudienceNumRatings','rtAudienceScore','titleSentiment']],
                      genres, left_index=True, right_index=True, how = 'inner')
moviesData = pd.merge(moviesData, directors, left_index=True, right_index=True, how = 'inner')
moviesData = pd.merge(moviesData, colors, left_index = True, right_index = True)
moviesData = pd.merge(moviesData, actors, left_index=True, right_index=True, how = 'inner')
moviesData = pd.merge(moviesData, tags[['tagSentiment']], left_index=True, right_index=True)
moviesData = pd.merge(moviesData, countries[['countryNum']], left_index=True, right_index=True, how = 'inner')
moviesData = moviesData[moviesData.realGenre != 'None']
moviesData = moviesData.dropna()

In [28]:
moviesData.head()

,title,year,rtAudienceNumRatings,rtAudienceScore,titleSentiment,realGenre,genreNum,directorName,directorNum,posterColor,color,colorNum,actorName,actorNum,tagSentiment,countryNum
1,Toy story,1995,102338,81,0.0,Comedy,1,John Lasseter,2027,dark red,"(104.392166836, 54.7272562127, 43.522889115)",5,Tom Hanks,7241,0.356944,66
3,Grumpy Old Men,1993,10489,66,0.1,Romance,5,Donald Petrie,1038,white,"(240.257754103, 245.865108686, 244.831228342)",11,Jack Lemmon,3122,0.100000,66
5,Father of the Bride Part II,1995,13761,64,0.0,Comedy,1,Charles Shyer,609,dark red,"(66.3822844645, 58.6215606039, 44.2912374771)",5,Steve Martin,6881,0.000000,66
6,Heat,1995,42785,92,0.0,Action,0,Michael Mann,2716,light blue,"(148.308032972, 178.875707128, 227.418005495)",6,Al Pacino,120,0.060606,66
7,Sabrina,1954,12812,87,0.0,Romance,5,Billy Wilder,398,black,"(11.6874206517, 10.4756312597, 11.2805579066)",0,Humphrey Bogart,3009,0.000000,66


Now to clean up the types of data

In [29]:
moviesData['rtAudienceNumRatings'] = moviesData.rtAudienceNumRatings.replace({'\\N':'0', 'NA': '0'})
moviesData['rtAudienceScore'] = moviesData.rtAudienceScore.replace({'\\N':'0', 'NA': '0'})

In [30]:
moviesData.rtAudienceNumRatings = moviesData.rtAudienceNumRatings.astype('int')
moviesData.rtAudienceScore = moviesData.rtAudienceScore.astype('int')
moviesData.real_genre = moviesData.realGenre.astype('category')
moviesData.directorName = moviesData.directorName.astype('category')
moviesData.poster_color = moviesData.posterColor.astype('category')
moviesData.actorName = moviesData.actorName.astype('category')

In [31]:
moviesData.dtypes

title                     object
year                       int64
rtAudienceNumRatings       int64
rtAudienceScore            int64
titleSentiment           float64
realGenre                 object
genreNum                    int8
directorName            category
directorNum                int16
posterColor               object
color                     object
colorNum                    int8
actorName               category
actorNum                   int16
tagSentiment             float64
countryNum                  int8
dtype: object

In [32]:
assert pd.notnull(moviesData.any().any())

## Saving Data

Last we save the data as a pickle named moviesData.

In [33]:
moviesData.to_pickle(prefix + 'moviesData.pkl')